In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model

def mlp(x, cf):
    x = L.Dense(cf["mlp_dim"], activation="gelu")(x)
    x = L.Dropout(cf["dropout_rate"])(x)
    x = L.Dense(cf["hidden_dim"])(x)
    x = L.Dropout(cf["dropout_rate"])(x)
    return x

def transformer_encoder(x, cf):
    skip_1 = x
    x = L.LayerNormalization()(x)
    x = L.MultiHeadAttention(
        num_heads=cf["num_heads"], key_dim=cf["hidden_dim"]
    )(x, x)
    x = L.Add()([x, skip_1])

    skip_2 = x
    x = L.LayerNormalization()(x)
    x = mlp(x, cf)
    x = L.Add()([x, skip_2])

    return x

def conv_block(x, num_filters, kernel_size=3):
    x = L.Conv2D(num_filters, kernel_size=kernel_size, padding="same")(x)
    x = L.BatchNormalization()(x)
    x = L.ReLU()(x)
    return x

def deconv_block(x, num_filters):
    x = L.Conv2DTranspose(num_filters, kernel_size=2, padding="same", strides=2)(x)
    return x

def build_unetr_2d(cf):
    """ Inputs """
    input_shape = (cf["image_size"], cf["image_size"], cf["num_channels"])
    inputs = L.Input(input_shape)

    # Resize the input image to the expected patch size
    x = L.Reshape((cf["num_patches"], cf["patch_size"] * cf["patch_size"] * cf["num_channels"]))(inputs)
    # x = L.Dense(cf["hidden_dim"])(x)
    
    # """ Inputs """
    # input_shape = (cf["num_patches"], cf["patch_size"]*cf["patch_size"]*cf["num_channels"])
    # inputs = L.Input(input_shape) ## (None, 256, 768)

    # """ Patch + Position Embeddings """
    patch_embed = L.Dense(cf["hidden_dim"])(x) ## (None, 256, 768)

    positions = tf.range(start=0, limit=cf["num_patches"], delta=1) ## (256,)
    pos_embed = L.Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions) ## (256, 768)
    x = patch_embed + pos_embed ## (None, 256, 768)

    """ Transformer Encoder """
    skip_connection_index = [3, 6, 9, 12]
    skip_connections = []

    for i in range(1, cf["num_layers"]+1, 1):
        x = transformer_encoder(x, cf)

        if i in skip_connection_index:
            skip_connections.append(x)

    """ CNN Decoder """
    z3, z6, z9, z12 = skip_connections

    ## Reshaping
    z0 = L.Reshape((cf["image_size"], cf["image_size"], cf["num_channels"]))(inputs)
    z3 = L.Reshape((cf["patch_size"], cf["patch_size"], cf["hidden_dim"]))(z3)
    z6 = L.Reshape((cf["patch_size"], cf["patch_size"], cf["hidden_dim"]))(z6)
    z9 = L.Reshape((cf["patch_size"], cf["patch_size"], cf["hidden_dim"]))(z9)
    z12 = L.Reshape((cf["patch_size"], cf["patch_size"], cf["hidden_dim"]))(z12)

    ## Decoder 1
    x = deconv_block(z12, 512)

    s = deconv_block(z9, 512)
    s = conv_block(s, 512)
    x = L.Concatenate()([x, s])

    x = conv_block(x, 512)
    x = conv_block(x, 512)

    ## Decoder 2
    x = deconv_block(x, 256)

    s = deconv_block(z6, 256)
    s = conv_block(s, 256)
    s = deconv_block(s, 256)
    s = conv_block(s, 256)

    x = L.Concatenate()([x, s])
    x = conv_block(x, 256)
    x = conv_block(x, 256)

    ## Decoder 3
    x = deconv_block(x, 128)

    s = deconv_block(z3, 128)
    s = conv_block(s, 128)
    s = deconv_block(s, 128)
    s = conv_block(s, 128)
    s = deconv_block(s, 128)
    s = conv_block(s, 128)

    x = L.Concatenate()([x, s])
    x = conv_block(x, 128)
    x = conv_block(x, 128)

    ## Decoder 4
    x = deconv_block(x, 64)

    s = conv_block(z0, 64)
    s = conv_block(s, 64)

    x = L.Concatenate()([x, s])
    x = conv_block(x, 64)
    x = conv_block(x, 64)

    """ Output """
    # outputs = L.Conv2D(11, kernel_size=1, padding="same", activation="sigmoid")(x)
    outputs = L.Conv2D(11, kernel_size=1, padding="same", activation="softmax")(x)


    return Model(inputs, outputs, name="UNETR_2D")

if __name__ == "__main__":
    config = {}
    config["image_size"] = 256
    config["num_layers"] = 12
    config["hidden_dim"] = 768
    config["mlp_dim"] = 3072
    config["num_heads"] = 12
    config["dropout_rate"] = 0.1
    config["num_patches"] = 256
    config["patch_size"] = 16
    config["num_channels"] = 3

    model = build_unetr_2d(config)
    model.summary()

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
import scipy.io
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger



""" Global parameters """
global IMG_H
global IMG_W
global NUM_CLASSES
global CLASSES
global COLORMAP

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

""" Load and split the dataset """
def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "img_small_data", "*")))[:1110]
    masks = sorted(glob(os.path.join(path, "mask_small_data", "*")))[:1110]

    split_size = int(split * len(images))

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


def get_colormap(path):
    mat_path = os.path.join(path, "multi_organ.mat")
    colormap = scipy.io.loadmat(mat_path)["multi_organ"]

    classes = [
        "Background",
        "Spleen",
        "Right kidney",
        "Left kidney",
        "Gallbladder",
        "Liver",
        "Stomach",
        "Aorta",
        "Inferior vena cava",
        "Portal vein",
        "pancreas"
    ]

    return classes, colormap

def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))
    x = x.astype(np.float32)
    return x

def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))

    #Masl processing
    output=[]
    # for i,color in enumerate(COLORMAP):
    #   cmap = np.all(np.equal(x, color), axis=-1)
    #   cv2.imwrite(f"cmap{i}.png", cmap*255)
    for color in COLORMAP:
      cmap = np.all(np.equal(x, color), axis=-1)
      output.append(cmap)


    output = np.stack(output, axis=-1)
    output = output.astype(np.uint8)

    return output


def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        x =read_image(x)
        y=read_mask(y)

        return x,y

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.uint8])
    image.set_shape([IMG_H, IMG_W, 3])
    mask.set_shape([IMG_H, IMG_W, NUM_CLASSES])

    return image, mask

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Hyperparameters """
    IMG_H = 256
    IMG_W = 256
    NUM_CLASSES = 11
#     input_shape = (IMG_H, IMG_W, 3)

    batch_size = 6
    lr = 1e-4
    num_epochs = 60

    config = {}
    config["image_size"] = 256
    config["num_layers"] = 12
    config["hidden_dim"] = 768
    config["mlp_dim"] = 3072
    config["num_heads"] = 12
    config["dropout_rate"] = 0.1
    config["num_patches"] = 256
    config["patch_size"] = 16
    config["num_channels"] = 3

    dataset_path = "/kaggle/input/small-data/small_data"
    model_path = os.path.join("files", "final_unter_model.h5")
    csv_path = os.path.join("files", "Unetr_Abdomen_data.csv")

    """ Loading the dataset """
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_x)}")

    """ Process the colormap """
    CLASSES, COLORMAP = get_colormap(dataset_path)

    print("COLOR:",len(COLORMAP))

    # read_mask(train_y[10])

    """ Dataset Pipeline """
    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

    """ Model """
    model = build_unetr_2d(config)
    # model.load_weights(model_path)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(lr)
    )
    # model.summary()

    """ Training """
    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=num_epochs,
        callbacks=callbacks
    )


In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
import scipy.io
from sklearn.model_selection import train_test_split
import tensorflow as tf
from PIL import Image
# from train import load_dataset, create_dir, get_colormap

""" Global parameters """
global IMG_H
global IMG_W
global NUM_CLASSES
global CLASSES
global COLORMAP

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

""" Load and split the dataset """
def load_dataset(path, split=0.2):
    images = sorted(glob(os.path.join(path, "img_small_data", "*")))[:1110]
    masks = sorted(glob(os.path.join(path, "mask_small_data", "*")))[:1110]

    split_size = int(split * len(images))

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


def get_colormap(path):
    mat_path = os.path.join(path, "multi_organ.mat")
    colormap = scipy.io.loadmat(mat_path)["multi_organ"]

    classes = [
        "Background",
        "Spleen",
        "Right kidney",
        "Left kidney",
        "Gallbladder",
        "Liver",
        "Stomach",
        "Aorta",
        "Inferior vena cava",
        "Portal vein",
        "pancreas"
    ]

    return classes, colormap

def read_image(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))
    x = x.astype(np.float32)
    return x

def read_mask(x):
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (IMG_W, IMG_H))

    #Masl processing
    output=[]
    # for i,color in enumerate(COLORMAP):
    #   cmap = np.all(np.equal(x, color), axis=-1)
    #   cv2.imwrite(f"cmap{i}.png", cmap*255)
    for color in COLORMAP:
      cmap = np.all(np.equal(x, color), axis=-1)
      output.append(cmap)


    output = np.stack(output, axis=-1)
    output = output.astype(np.uint8)

    return output


def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()

        x =read_image(x)
        y=read_mask(y)

        return x,y

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.uint8])
    image.set_shape([IMG_H, IMG_W, 3])
    mask.set_shape([IMG_H, IMG_W, NUM_CLASSES])

    return image, mask

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset


def grayscale_to_rgb(mask, classes, colormap):
    h, w, _ = mask.shape
    mask = mask.astype(np.int32)
    output = []

    for i, pixel in enumerate(mask.flatten()):
        output.append(colormap[pixel])

    output = np.reshape(output, (h, w, 3))
    return output

def save_results(image, mask, pred, save_image_path):
    # print(image.shape,mask.shape,pred.shape)
    h, w, _ = image.shape
    line = np.ones((h, 10, 3)) * 255

    pred = np.expand_dims(pred, axis=-1)
    pred = grayscale_to_rgb(pred, CLASSES, COLORMAP)

    # Ensure both images have the same shape
    assert image.shape == mask.shape, "Images must have the same shape"
    assert image.shape == pred.shape

    # Blend the images using the alpha parameter
    alpha = 0.5
    blended_image1 = alpha * image + (1 - alpha) * mask
    blended_image2 = alpha * image + (1 - alpha) * pred
    # alpha = 0.5
    # blended_image1 = Image.blend(image, mask, alpha)
    # blended_image2 = Image.blend(image, pred, alpha)

    cat_images = np.concatenate([image, line, mask, line, pred, line, blended_image1, blended_image2], axis=1)
    cv2.imwrite(save_image_path, cat_images)


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Directory for storing files """
    create_dir("results2/predictions")

    """ Hyperparameters """
    IMG_H = 256
    IMG_W = 256
    NUM_CLASSES = 11
    dataset_path = "/kaggle/input/small-data/small_data"
    # dataset_path = "/content/drive/MyDrive/human"
    # model_path = os.path.join("files", "model1.h5")
    model_path = "/kaggle/working/files/final_unter_model.h5"

    """ Colormap """
    CLASSES, COLORMAP = get_colormap(dataset_path)

    """ Model """
    model = tf.keras.models.load_model(model_path)
    # model.summary()

    """ Load the dataset """
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
    print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_y)}")
    print("")

    # Prediction and Evaluation

    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
      name = x.split("/")[-1].split(".")[0]

      """ Reading the image """
      image = cv2.imread(x, cv2.IMREAD_COLOR)
      image = cv2.resize(image, (IMG_W, IMG_H))
      image_x = image
      # image = image/255.0
      image = np.expand_dims(image, axis=0)

      """ Reading the mask """
      mask = cv2.imread(y, cv2.IMREAD_COLOR)
      mask = cv2.resize(mask, (IMG_W, IMG_H))
      mask_x = mask
      onehot_mask = []
      for color in COLORMAP:
          cmap = np.all(np.equal(mask, color), axis=-1)
          onehot_mask.append(cmap)
      onehot_mask = np.stack(onehot_mask, axis=-1)
      onehot_mask = np.argmax(onehot_mask, axis=-1)
      onehot_mask = onehot_mask.astype(np.int32)

      """ Prediction """
      pred = model.predict(image, verbose=0)[0]
      pred = np.argmax(pred, axis=-1)
      pred = pred.astype(np.float32)

      """ Saving the prediction """
      save_image_path = f"results2/predictions/{name}.png"
      save_results(image_x, mask_x, pred, save_image_path)

      """ Flatten the array """
      onehot_mask = onehot_mask.flatten()
      pred = pred.flatten()

      labels = [i for i in range(NUM_CLASSES)]

      """ Calculating the metrics values """
      f1_value = f1_score(onehot_mask, pred, labels=labels, average=None, zero_division=0)
      jac_value = jaccard_score(onehot_mask, pred, labels=labels, average=None, zero_division=0)

      SCORE.append([f1_value, jac_value])


    # """ Metrics values """
    # score = np.array(SCORE)
    # score = np.mean(score, axis=0)

    # f = open("files/score.csv", "w")
    # f.write("Class,F1,Jaccard\n")

    # l = ["Class", "F1", "Jaccard"]
    # print(f"{l[0]:15s} {l[1]:10s} {l[2]:10s}")
    # print("-"*35)

    # for i in range(score.shape[1]):
    #     class_name = CLASSES[i]
    #     f1 = score[0, i]
    #     jac = score[1, i]
    #     dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
    #     print(dstr)
    #     f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f}\n")

    # print("-"*35)
    # class_mean = np.mean(score, axis=-1)
    # class_name = "Mean"
    # f1 = class_mean[0]
    # jac = class_mean[1]
    # dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
    # print(dstr)
    # f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f}\n")

    # f.close()

    """ Metrics values """
    score = np.array(SCORE)
    score = np.mean(score, axis=0)

    # Calculate accuracy using true labels and predicted labels
    accuracy = accuracy_score(onehot_mask, pred.flatten())

    f = open("files/score.csv", "w")
    f.write("Class,F1,Jaccard,Accuracy\n")

    l = ["Class", "F1", "Jaccard", "Accuracy"]
    print(f"{l[0]:15s} {l[1]:10s} {l[2]:10s} {l[3]:10s}")
    print("-" * 50)

    for i in range(score.shape[1]):
        class_name = CLASSES[i]
        f1 = score[0, i]
        jac = score[1, i]
        dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f} - {accuracy:1.5f}"
        print(dstr)
        f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f},{accuracy:1.5f}\n")

    print("-" * 50)
    class_mean = np.mean(score, axis=-1)
    class_name = "Mean"
    f1 = class_mean[0]
    jac = class_mean[1]
    dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f} - {accuracy:1.5f}"
    print(dstr)
    f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f},{accuracy:1.5f}\n")

    f.close()



